In [1]:
# ============================================================
# Bayesian Network: Burglary–Earthquake–Alarm–Calls
# ============================================================

# ------------------------------------------------------------
# PRIOR PROBABILITIES
# ------------------------------------------------------------
P_B = 0.001      # Burglary
P_E = 0.002      # Earthquake

# ------------------------------------------------------------
# CPT: P(A | B, E)
# ------------------------------------------------------------
P_A_given = {
    (True,  True): 0.95,
    (True,  False): 0.94,
    (False, True): 0.29,
    (False, False): 0.001
}

# Helper function to compute P(A)
def marginal_P_A():
    return (
        P_A_given[(True, True)]   * P_B * P_E +
        P_A_given[(True, False)]  * P_B * (1 - P_E) +
        P_A_given[(False, True)]  * (1 - P_B) * P_E +
        P_A_given[(False, False)] * (1 - P_B) * (1 - P_E)
    )

# ============================================================
# QUESTION 1: Independence without evidence
# ============================================================
print("QUESTION 1 — Are B and E independent without evidence?\n")

P_B_and_E = P_B * P_E  # because BN topology says they are independent a priori

print(f"P(B) = {P_B}")
print(f"P(E) = {P_E}")
print(f"P(B ∧ E) = P(B) * P(E) = {P_B_and_E:.8f}")
print("\nTopological reasoning: B → A ← E is a collider.")
print("Collider unobserved → path blocked → B and E are d-separated → Independent.\n")


# ============================================================
# QUESTION 2: Conditional independence given Alarm=True
# ============================================================
print("\nQUESTION 2 — Are B and E independent given Alarm = True?\n")

P_A = marginal_P_A()
print(f"P(A) = {P_A:.8f}")

# ------------------------------------------------------------
# Compute P(B | A)
# ------------------------------------------------------------
numerator_B_given_A = (
    P_B * P_E * P_A_given[(True, True)] +
    P_B * (1 - P_E) * P_A_given[(True, False)]
)
P_B_given_A = numerator_B_given_A / P_A
print(f"P(B | A) = {P_B_given_A:.6f}")

# ------------------------------------------------------------
# Compute P(B | A, E=True)
# ------------------------------------------------------------
num = P_B * P_A_given[(True, True)]
den = (
    P_B * P_A_given[(True, True)] +
    (1 - P_B) * P_A_given[(False, True)]
)
P_B_given_A_E_true = num / den
print(f"P(B | A, E=True) = {P_B_given_A_E_true:.6f}")

# ------------------------------------------------------------
# Compute P(B | A, E=False)
# ------------------------------------------------------------
num = P_B * P_A_given[(True, False)]
den = (
    P_B * P_A_given[(True, False)] +
    (1 - P_B) * P_A_given[(False, False)]
)
P_B_given_A_E_false = num / den
print(f"P(B | A, E=False) = {P_B_given_A_E_false:.6f}")

# ------------------------------------------------------------
# Conclusion
# ------------------------------------------------------------
print("\nCONCLUSION:")
print("If P(B | A) ≠ P(B | A, E), then B and E are NOT conditionally independent given A.")
print(f"P(B | A)          = {P_B_given_A:.6f}")
print(f"P(B | A, E=True)  = {P_B_given_A_E_true:.6f}")
print(f"P(B | A, E=False) = {P_B_given_A_E_false:.6f}")
print("\nTherefore: Burglary and Earthquake become dependent given Alarm=True (explaining away).")


QUESTION 1 — Are B and E independent without evidence?

P(B) = 0.001
P(E) = 0.002
P(B ∧ E) = P(B) * P(E) = 0.00000200

Topological reasoning: B → A ← E is a collider.
Collider unobserved → path blocked → B and E are d-separated → Independent.


QUESTION 2 — Are B and E independent given Alarm = True?

P(A) = 0.00251644
P(B | A) = 0.373551
P(B | A, E=True) = 0.003268
P(B | A, E=False) = 0.484786

CONCLUSION:
If P(B | A) ≠ P(B | A, E), then B and E are NOT conditionally independent given A.
P(B | A)          = 0.373551
P(B | A, E=True)  = 0.003268
P(B | A, E=False) = 0.484786

Therefore: Burglary and Earthquake become dependent given Alarm=True (explaining away).


In [2]:
import math
import random
from typing import List, Tuple, Any, Optional

# Deterministic RNG for reproducibility
random.seed(2025)

LANES = 8
LANE_PATTERN = [4, 5, 3, 6, 2, 7, 1, 8]

# Round sizes
ROUND_SIZES = {"R0": 80, "R1": 40, "R2": 24, "R3": 8}

# -------------------------------
# Domain objects
# -------------------------------
class Athlete:
    def __init__(self, athlete_id: int, name: str, base_ability: float):
        self.id = athlete_id
        self.name = name
        self.base_ability = base_ability  # lower is better
        self.last_round_time: Optional[float] = None

class Assignment:
    def __init__(self, round_name: str, heat_index: int, lane: int, athlete: Athlete, order_rank: int):
        self.round = round_name
        self.heat_index = heat_index
        self.lane = lane
        self.athlete = athlete
        self.order_rank = order_rank

class Result:
    def __init__(self, assignment: Assignment, time: float, place: int):
        self.assignment = assignment
        self.time = time
        self.place = place

# -------------------------------
# Bayesian reasoning helper
# -------------------------------
def bayesian_qualification_probability(athlete, time: float, place: int) -> float:
    """
    Compute probability of qualification given race time and place.
    """
    # Prior ability categories (Fast, Medium, Slow)
    priors = {"Fast": 0.3, "Medium": 0.5, "Slow": 0.2}
    means = {"Fast": 10.0, "Medium": 11.0, "Slow": 12.0}
    std_dev = 0.2

    # Likelihood of observed time given ability (Gaussian approximation)
    likelihoods = {}
    for ability, mean in means.items():
        exponent = -((time - mean) ** 2) / (2 * std_dev ** 2)
        likelihoods[ability] = math.exp(exponent)

    # Posterior over ability
    post_ability = {}
    total = 0.0
    for ability in priors:
        post_ability[ability] = priors[ability] * likelihoods[ability]
        total += post_ability[ability]
    for ability in post_ability:
        post_ability[ability] /= total

    # Heat result probability (simplified: top 2 auto-qualify)
    heat_win_prob = 1.0 if place <= 2 else 0.0

    # Qualification probability
    if heat_win_prob == 1.0:
        return 1.0
    else:
        if time < 11.0:
            return 0.5
        else:
            return 0.1

# -------------------------------
# Generation and helpers
# -------------------------------
def generate_unseeded_athletes(n: int) -> List[Athlete]:
    athletes = []
    for i in range(n):
        ability = 10.8 + random.gauss(0, 0.6)
        ability = max(9.75, min(12.6, ability))
        athletes.append(Athlete(i + 1, f"Athlete_{i+1}", round(ability, 3)))
    return athletes

def serpentine_groups(entities: List[Any], size: int) -> List[List[Any]]:
    n = len(entities)
    heats = math.ceil(n / size)
    groups = [[] for _ in range(heats)]
    idx = 0
    direction = 1
    while idx < n:
        if direction == 1:
            for h in range(heats):
                if idx < n and len(groups[h]) < size:
                    groups[h].append(entities[idx]); idx += 1
            direction = -1
        else:
            for h in reversed(range(heats)):
                if idx < n and len(groups[h]) < size:
                    groups[h].append(entities[idx]); idx += 1
            direction = 1
    return groups

def assign_lanes(group: List[Athlete], round_name: str, heat_index: int, key_func) -> List[Assignment]:
    ranked = sorted(group, key=key_func)
    assignments = []
    for i, athlete in enumerate(ranked):
        lane = LANE_PATTERN[i] if i < len(LANE_PATTERN) else (i+1)
        assignments.append(Assignment(round_name, heat_index, lane, athlete, i + 1))
    return assignments

def simulate_heat(assignments: List[Assignment], noise_std: float = 0.08) -> List[Result]:
    results = []
    for a in assignments:
        noise = random.gauss(0, noise_std)
        simulated = max(9.5, a.athlete.base_ability + noise)
        res = Result(a, round(simulated, 3), place=0)
        results.append(res)

    # sort by time ascending; deterministic tie-break: athlete id
    results.sort(key=lambda r: (r.time, r.assignment.athlete.id))
    for i, r in enumerate(results, start=1):
        r.place = i
        r.assignment.athlete.last_round_time = r.time
        # NEW: Bayesian qualification probability
        q_prob = bayesian_qualification_probability(r.assignment.athlete, r.time, r.place)
        print(f"    {i}. {r.assignment.athlete.name} — {r.time}s (Lane {r.assignment.lane}) "
              f"→ P(Qualify)={q_prob:.2f}")
    return results

# -------------------------------
# Progression logic
# -------------------------------
def sort_with_tie_break(candidates: List[Tuple[Athlete, float]]) -> List[Tuple[Athlete, float]]:
    def key(x):
        ath, t = x
        prev = ath.last_round_time if ath.last_round_time is not None else float('inf')
        return (t, prev, ath.id)
    return sorted(candidates, key=key)

def progress_by_winners_and_time(all_heat_results: List[List[Result]], target_size: int) -> List[Athlete]:
    winners = [heat[0].assignment.athlete for heat in all_heat_results]
    remaining = []
    for heat in all_heat_results:
        for r in heat[1:]:
            remaining.append((r.assignment.athlete, r.time))
    remaining_sorted = sort_with_tie_break(remaining)
    need = target_size - len(winners)
    qualifiers = winners + [ath for ath, _ in remaining_sorted[:need]]
    # deduplicate while preserving order
    uniq, seen = [], set()
    for a in qualifiers:
        if a.id not in seen:
            uniq.append(a); seen.add(a.id)
    if len(uniq) != target_size:
        raise ValueError("Progression size mismatch; check qualification rule.")
    return uniq

# -------------------------------
# Printing helpers
# -------------------------------
def print_round(all_assignments: List[List[Assignment]], all_results: List[List[Result]], round_name: str):
    for h_idx, (assigns, results) in enumerate(zip(all_assignments, all_results), start=1):
        print(f"\nRound {round_name} — Heat {h_idx}")
        print("  Assignments (seed/order_rank):")
        for a in sorted(assigns, key=lambda x: x.lane):
            print(f"    Lane {a.lane}: {a.athlete.name} (base {a.athlete.base_ability}s) rank {a.order_rank}")
        print("  Results (simulated times + Bayesian qualification):")
        # results already printed inside simulate_heat

def print_progression(from_round: str, to_round: str, qualifiers: List[Athlete]):
    print(f"\nProgression from {from_round} -> {to_round} (total qualifiers: {len(qualifiers)}):")
    for i, a in enumerate(qualifiers, start=1):
        print(f"  {i}. {a.name} — last_time={a.last_round_time}s base={a.base_ability}s")
    print()

# -------------------------------
# Round orchestration
# -------------------------------
def round_pipeline(athletes: List[Athlete], round_name: str, key_func, noise_std=0.08):
    groups = serpentine_groups(athletes, LANES)
    all_assignments, all_results = [], []
    for idx, group in enumerate(groups, start=1):
        assigns = assign_lanes(group, round_name, idx, key_func=key_func)
        results = simulate_heat(assigns, noise_std=noise_std)
        all_assignments.append(assigns)
        all_results.append(results)
    print_round(all_assignments, all_results, round_name)
    return all_assignments, all_results

def run_meet():
    # R0
    athletes_R0 = generate_unseeded_athletes(ROUND_SIZES["R0"])
    print("\n========== R0 (Unseeded) ==========")
    R0_assigns, R0_results = round_pipeline(
        athletes_R0, "R0", key_func=lambda a: a.base_ability, noise_std=0.085
    )

    # Progress to R1 (40)
    R1_field = progress_by_winners_and_time(R0_results, ROUND_SIZES["R1"])
    print_progression("R0", "R1", R1_field)

    # R1 seeded heats
    print("\n========== R1 (Seeded Heats) ==========")
    R1_assigns, R1_results = round_pipeline(
        R1_field, "R1", key_func=lambda a: a.base_ability, noise_std=0.075
    )

    # Progress to R2 (24)
    R2_field = progress_by_winners_and_time(R1_results, ROUND_SIZES["R2"])
    print_progression("R1", "R2", R2_field)

    # R2 semifinals
    print("\n========== R2 (Semifinals) ==========")
    R2_assigns, R2_results = round_pipeline(
        R2_field,
        "R2",
        key_func=lambda a: (a.last_round_time if a.last_round_time is not None else a.base_ability),
        noise_std=0.07,
    )

    # Progress to R3 (8)
    R3_field = progress_by_winners_and_time(R2_results, ROUND_SIZES["R3"])
    print_progression("R2", "R3", R3_field)

    # R3 final
    print("\n========== R3 (Final) ==========")
    R3_assigns, R3_results = round_pipeline(
        R3_field,
        "R3",
        key_func=lambda a: (a.last_round_time if a.last_round_time is not None else a.base_ability),
        noise_std=0.065,
    )

    # Final results print (R3)
    print("\nFinal (R3) results:")
    for r in R3_results[0]:
        print(f"  {r.place}. {r.assignment.athlete.name} — {r.time}s (Lane {r.assignment.lane})")

if __name__ == "__main__":
    run_meet()


SyntaxError: unterminated string literal (detected at line 196) (933650295.py, line 196)